In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd
import pvlib
from pvlib.location import Location
from sympy import *
from sympy.abc import x,y,z,u,v,w,t
from dtumathtools import *
init_printing()

In [12]:
timezone = "Europe/Copenhagen"
start_date = "2024-06-01"
end_date = "2024-07-04"
delta_time = "H"  # "Min", "H", 

# Definition of Location object. Coordinates and elevation of Amager, Copenhagen (Denmark)
site = Location(
    55.785469, 12.519103, timezone, 10, "DTU (DK)"
)  # latitude, longitude, time_zone, altitude, name

# Definition of a time range of simulation
times = pd.date_range(
    start_date + " 00:00:00", end_date + " 23:59:00", freq=delta_time, tz=timezone
)
#print(times)

sunpos = site.get_solarposition(times)

sunpos.apparent_elevation.max(),display(sunpos)

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2024-06-01 00:00:00+02:00,100.814547,100.814547,-10.814547,-10.814547,344.046865,2.174048
2024-06-01 01:00:00+02:00,102.112956,102.112956,-12.112956,-12.112956,358.161652,2.167611
2024-06-01 02:00:00+02:00,101.336428,101.336428,-11.336428,-11.336428,12.328847,2.161163
2024-06-01 03:00:00+02:00,98.552882,98.552882,-8.552882,-8.552882,26.148392,2.154702
2024-06-01 04:00:00+02:00,93.988408,93.988408,-3.988408,-3.988408,39.350649,2.148229
...,...,...,...,...,...,...
2024-07-04 19:00:00+02:00,69.286819,69.330727,20.713181,20.669273,280.441160,-4.570209
2024-07-04 20:00:00+02:00,77.341616,77.413914,12.658384,12.586086,292.156895,-4.577508
2024-07-04 21:00:00+02:00,84.691122,84.847955,5.308878,5.152045,303.960634,-4.584797
2024-07-04 22:00:00+02:00,91.296350,91.296350,-1.296350,-1.296350,316.200777,-4.592077


(57.59065985962742, None)

In [16]:
sunpos = site.get_solarposition(times)
def solar_flux(i):
    #Defining angles in radians
    alpha = np.deg2rad(sunpos.apparent_elevation)
    azimuth = np.deg2rad(sunpos.azimuth)
    vector = (Matrix([cos(azimuth[i]),sin(azimuth[i]),tan(alpha[i])]))
    return(vector.normalized()*1100)
a = solar_flux(57)
display((a).evalf())
sqrt(a[0]**2 +a[1]**2 + a[2]**2)
     


C:\Users\emils\AppData\Local\Temp\ipykernel_36064\4195698290.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vector = (Matrix([cos(azimuth[i]),sin(azimuth[i]),tan(alpha[i])]))


⎡-158.861415765889⎤
⎢                 ⎥
⎢898.119832268822 ⎥
⎢                 ⎥
⎣614.933994398001 ⎦

In [4]:
#rn must use radians, maybe change so can be used as degrees?
def panel_normal(azimuth,alpha):
    vector = (Matrix([cos(azimuth),sin(azimuth),tan(alpha)]))
    return(vector.normalized())
b = panel_normal(np.deg2rad(180),np.deg2rad(45))
b.evalf()

⎡ -0.707106781186548 ⎤
⎢                    ⎥
⎢8.65956056235493e-17⎥
⎢                    ⎥
⎣ 0.707106781186547  ⎦

In [20]:
max(0,a.dot(b))*1*1*0.5